In [1]:
from agents.dpm_ppo import Agent, preprocess
from trading_env.environment import TradingEnv

In [2]:
import dill
import numpy as np

In [3]:
yf_file = "./data/archive_data/yf_data.dill"
with open(yf_file,'rb') as dill_file:
    yf_df = dill.load(dill_file)
    

In [4]:
symbols = yf_df['Symbol'].unique()
stocks = []
for symbol in symbols:
    df = yf_df[yf_df['Symbol'] == symbol]
    stocks.append(df.iloc[:,2:6].to_numpy())
stocks = np.array(stocks)

In [5]:
n_stocks = stocks.shape[0]
window = 64
n_stock_feats = stocks.shape[2]

In [6]:
agent = Agent(n_stocks,n_stock_feats)

In [7]:
env = TradingEnv(stocks)
obs = env.reset()

In [8]:
obs.shape

(21, 64, 4)

In [9]:
agent.actor(np.array([obs]))

<tf.Tensor: shape=(1, 22), dtype=float32, numpy=
array([[0.07272587, 0.04423793, 0.04411047, 0.04415678, 0.04446036,
        0.04411047, 0.04411047, 0.04411047, 0.04411047, 0.04437022,
        0.04411047, 0.04420298, 0.04411047, 0.04418852, 0.04411047,
        0.04411047, 0.04411047, 0.04411047, 0.04411047, 0.04411047,
        0.04411047, 0.04411047]], dtype=float32)>

In [10]:
x = np.array([obs])
agent.act(obs)

array([[0.07272587, 0.04423793, 0.04411047, 0.04415678, 0.04446036,
        0.04411047, 0.04411047, 0.04411047, 0.04411047, 0.04437022,
        0.04411047, 0.04420298, 0.04411047, 0.04418852, 0.04411047,
        0.04411047, 0.04411047, 0.04411047, 0.04411047, 0.04411047,
        0.04411047, 0.04411047]], dtype=float32)

In [11]:
all_aloss = []
all_closs = []
rewards = []
states = []
actions = []
probs = []
dones = []
values = []

In [12]:
for e in range(128):
   
    action = agent.act(obs)
    value = agent.critic(np.array([obs])).numpy()
    next_state, reward, done, _ = env.step(action)
    dones.append(1-done)
    rewards.append(reward)
    states.append(obs)
    #actions.append(tf.one_hot(action, 2, dtype=tf.int32).numpy().tolist())
    actions.append(action)
    prob = agent.actor(np.array([obs]))
    probs.append(prob[0])
    values.append(value[0][0])
    obs = next_state
    if done:
      env.reset()

In [13]:
value = agent.critic(np.array([obs])).numpy()
values.append(value[0][0])
np.reshape(probs, (len(probs),22))
probs = np.stack(probs, axis=0)

In [14]:
np.array(probs).shape

(128, 22)

In [15]:
states, actions,returns, adv  = preprocess(states, actions, rewards, dones, values, 1)

In [16]:
for epocs in range(10):
      al,cl = agent.learn(states, actions, adv, probs, returns)

In [17]:
print(states.shape)
print(actions.shape)
print(adv.shape)
print(probs.shape)
print(returns.shape)

(128, 21, 64, 4)
(128, 1, 22)
(128,)
(128, 22)
(128,)


In [18]:
agent.actor(np.array([states[0]]))

<tf.Tensor: shape=(1, 22), dtype=float32, numpy=
array([[0.07279531, 0.04415259, 0.04415259, 0.04415259, 0.04415259,
        0.04415259, 0.04415259, 0.04415259, 0.04415259, 0.04415259,
        0.04415259, 0.04415259, 0.04415259, 0.04415259, 0.04415259,
        0.04415259, 0.04415259, 0.04415259, 0.04415259, 0.04415259,
        0.04415259, 0.04415259]], dtype=float32)>

In [19]:
states[0].shape

(21, 64, 4)

In [20]:
np.array([states[0]]).shape

(1, 21, 64, 4)